<a href="https://colab.research.google.com/github/johngrahamreynolds/QuantumComputing/blob/main/canonical_algorithms/deutsch%E2%80%93jozsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The Deutsch and Deutsch–Jozsa algorithms
#### This Colab implements the Deutsch algorithm using Cirq before discussing its generalization, the Deutsch–Jozsa algorithm

###### johngrahamreynolds@gmail.com

First we install Cirq and import relevant packages to our workspace

In [1]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq

import random

installing cirq...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 285.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 5.5 MB/s eta 0:0

## Introduction and motivation for these algorithms

Given a black box function f: {0,1}^n -> {0,1}, promised to be balanced or constant,
we can show that quantum algorithms can determine its global nature (that is, constant or balanced) much faster than classical algotithms.
A constant function returns either 0 or 1 for all inputs while a balanced function returns half 1s, half 0s.
We call the function an oracle, and send it information to compute the value of the function for an input.
We refer to this as querying the oracle.

In the case of the Deutsch and the Deutsch-Jozsa algorithms, we can determine the global
property of the function with just a single query to the oracle by desinging a
quantum circuit that utilizes quantum parallelism and a phase kickback trick to encode the global property.
The phase kickback trick utilizes quantum intereference, either constructive or desctructive, when we measure.

A classical computer does not have the same computational ability. The minimum number of queries to the oracle to discern the function's global property on a classical computer is garuanteed to be more than the number of queries conducted by quantum algorithms.
In the case of the f: {0,1} -> {0,1}, a classical algorithm needs a minmum 2 queries to the oracle while
in the case of f: {0,1}^n -> {0,1}, a classical algorithm needs a minimum of 2^n/2 + 1 queries to the oracle.
In both cases, the quantum Deutsch and Deutsch-Jozsa algorithms, respectively, need only a single query.
This demonstrates in the generalized case that the quantum Deutsch-Jozsa algorithm provides exponential speedup.

## The Deutsch algorithm

Simplification to the case of n = 1 gives us four possible functions represented by the truth table below

| x | f_0(x) | f_1(x) | f_x(x) | f_notx(x) |
|---|--------|--------|--------|-----------|
| 0 |    0   |  1     |   0    |  1        |
| 1 |     0  |    1   |   1    |   0       |

The global nature of this function, whichever is chosen, can be found with just a single query to the oracle as discussed above.

### Problem Statement

Bob and Alice wish to see how fast they can solve a problem together while Bob hides information from Alice. Bob tells Alice that he is going to construct a function of the type described above. That is, he constructs a function of the form f: {0,1}^n -> {0,1} and promises Alice that it will be either constant or balanced. Bob asks Alice to discern the global propety of the function in as few queries as possible. She can query his black box function by sending it n bits of information each time. The pair agree to the case of n=1 to begin.

Alice has read "Mike and Ike's" book Quantum Computation and Quantum Information and decides that she can build a quantum circuit to implement Deutsch's algorithm to solve this problem.


## Implementation of the Deutsch algorithm

In [2]:
# First, Bob builds a custom gate to encode the operation of his blackbox function to be used in the Alice's circuit
# The blackbox oracle, unbeknowst to the Alice, knows whether the function is constant or balanced based on Bob's choice
# Given the case of n=1, there are 4 functions that can exist as exhibited in the truth table above
# Bob thus encodes his blackbox to operate in one of 4 ways on the qubit that Alice sends into his function. His choice of function determines which computation is run
# Note that while Bob is aware of the function's global property, he only computes the function's values for the n (qu)bits Alice sends him. He does not tell her the answer explicitly
class bobs_deutsch_oracle(cirq.Gate):

  def __init__(self, function_type: str):
        super(bobs_deutsch_oracle, self)
        self.function_type = function_type

  def _num_qubits_(self) -> int:
    return 2

  def _decompose_(self, qubits):
    if self.function_type == "constant_0":
      yield cirq.IdentityGate(2).on(qubits[0], qubits[1])
    elif self.function_type == "constant_1":
      yield cirq.X(qubits[1])
    elif self.function_type == "balanced_x":
      yield cirq.CNOT(qubits[0], qubits[1])
    elif self.function_type == "balanced_notx":
      yield [cirq.CNOT(qubits[0], qubits[1]), cirq.X(qubits[1])]

  def _circuit_diagram_info_(self, args):
        return ["deutsch"] * self._num_qubits_()


In [3]:
# Now Alice designs her quantum circuit
# She builds it to interact with the black box oracle that Bob will configure based on his choice of the function
# Wisely, she designs the circuit in the form of Deutsch's alrogithm to employ quantum parallelism, a phase kickback trick, and finally quantum interference
# By designing her algorithm this way, she need query Bob's blackbox function within her circuit only a single time to discern the function's global nature
def alices_deutsch_algorithm(function_type: str, repetitions: int = 1) -> int:

  query_register = cirq.LineQubit(0)
  answer_qubit = cirq.LineQubit(1)

  # flip the answer qubit to |1>
  moment_0 = cirq.Moment(cirq.X(answer_qubit))
  # apply Hadamard gate to the qubit in the query register
  moment_1 = cirq.Moment(cirq.H.on_each([query_register, answer_qubit]))
  # apply the quantum deutsch oracle
  moment_2 = [cirq.Moment(bobs_deutsch_oracle(function_type).on(query_register, answer_qubit))]
  # apply Hadamard again to the qubit in the query register
  moment_3 = cirq.Moment(cirq.H.on_each(query_register))
  # measure the query registrer
  moment_4 = cirq.Moment(cirq.measure(query_register, key='result'))

  circuit = cirq.Circuit((moment_0, moment_1, moment_2, moment_3, moment_4))

  print(circuit)
  sim = cirq.Simulator()
  return sim.run(circuit, repetitions=repetitions)


In [4]:
# create the list of possible black box functions for the oracle
possible_functions = ["constant_0", "constant_1", "balanced_x", "balanced_notx"]
# Alice's dictionary that intelligently decodes her quantum algorithm's results
decoding = {0: "constant", 1: "balanced"}

# Bob picks one of the possible functions at random
bobs_choice = random.choice(possible_functions)
print("Bob has randomly chosen the " + bobs_choice + " function.\n")

# Alice runs her algorithm with a single query to the oracle to discern the global nature of the function
print("Making intelligent use of Bob's black box oracle, Alice constructs her quantum ciruit as so: \n")
response = alices_deutsch_algorithm(bobs_choice)
print(f"\nShe runs her quantum algorithm and records a final qubit query register |(f(0) + f(1) mod 2)> in the state: |{response.measurements['result'][0][0]}>.\n")
print(f"She determines immediately, with just one query to the oracle, that the function is {decoding[response.measurements['result'][0][0]].upper()}.\n")

Bob has randomly chosen the balanced_notx function.

Making intelligent use of Bob's black box oracle, Alice constructs her quantum ciruit as so: 

0: ───────H───deutsch───H───M('result')───
              │
1: ───X───H───deutsch─────────────────────

She runs her quantum algorithm and records a final qubit query register |(f(0) + f(1) mod 2)> in the state: |1>.

She determines immediately, with just one query to the oracle, that the function is BALANCED.



In [5]:
# Indeed, one might wonder whether Alice just got lucky? Did she just have a high probabilty of being correct?
# The answer is no. She has engineered her algorithm to be fully deterministic
# We can illustrate this as so:
# Allow Bob to again randomly choose a function, and this time we allow Alice to run her algorithm #repetitions times
# We find that Alice is correct in all #repetitions attempts. Exponentially increasing the number of repeititions will never change any single output of the algorithm
repetitions = 15
# repetitions = 500
bobs_choice = random.choice(possible_functions)
print("This time, Bob has randomly chosen the " + bobs_choice + " function.\n")

print("Again, Alice's quantum ciruit is: \n")

response_2 = alices_deutsch_algorithm(bobs_choice, repetitions=repetitions)
print(f"\nAlice runs her algorithm {repetitions} times and determines in each case that that the function is:")
for i in range(repetitions):
  print(decoding[response_2.measurements['result'][0][0]].upper())

This time, Bob has randomly chosen the constant_0 function.

Again, Alice's quantum ciruit is: 

0: ───────H───deutsch───H───M('result')───
              │
1: ───X───H───deutsch─────────────────────

Alice runs her algorithm 15 times and determines in each case that that the function is:
CONSTANT
CONSTANT
CONSTANT
CONSTANT
CONSTANT
CONSTANT
CONSTANT
CONSTANT
CONSTANT
CONSTANT
CONSTANT
CONSTANT
CONSTANT
CONSTANT
CONSTANT


## The Deutsch-Jozsa algorithm

For the general case of any integer n greater than one, the Deutsch-Jozsa algorithm can still solve this problem with only a single query to the oracle. We expand our query register to n qubits, but otherwise the gates in our quantum circuit are identical with the generalization that the Deutsch (Deutsch-Jozsa) gate acts on the entire query register. Thus, one must be very careful to consider all possible combinations for the available balanced and constant functions while constructing the inner workings of the blackbox.